In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
df = pd.read_csv("data/hotel_csv_data.csv")

In [3]:
df.sample(n=5)

,101,NK,1,NUID065253359,1.1,071623 113711 101,11:37 AM 07/16/2023,2,1.2,01:37 PM 07/16/2023,...,41.08.4,0.00.4,0.00.5,Unnamed: 54,F.2,F.3,Unnamed: 57,3,7.00%6.00%4.000%17.000,0000
4291,108,K,1,TX18081731,1,041623 045120 108,04:51 PM 04/16/2023,1,1.0,05:51 PM 04/16/2023,...,25.0,0.0,0.0,NaN,F,F,NaN,115,7.00%6.00%4.000%17.000,0
1528,106,K,1,TX42777902,1,071523 030032 106,03:00 AM 07/15/2023,N,1.0,11:00 AM 07/15/2023,...,0.0,0.0,0.0,NaN,F,F,NaN,2,7.00%6.00%4.000%17.000,0
2222,126,K,1,TX46214137,1,022723 103625 126,2/27/2023 10:36:00 PM,N,1.0,2/28/2023 11:00:00 AM,...,0.0,0.0,0.0,NaN,F,F,NaN,1,17.000,0
2947,112,K,1,NUID204116328,1,021823 080134 112R,08:01 PM 02/18/2023,2,1.0,08:11 PM 02/18/2023,...,0.0,0.0,0.0,NaN,F,F,NaN,11,7.00%6.00%4.000%17.000,0
1743,118,NQQ,1,AR909112201,1,031623 110000 118,11:00 AM 03/16/2023,N,1.0,11:00 AM 03/17/2023,...,0.0,0.0,0.0,NaN,F,F,NaN,2,7.00%6.00%4.000%17.000,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6665 entries, 0 to 6664
Data columns (total 61 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   101                     6665 non-null   int64  
 1   NK                      6665 non-null   object 
 2   1                       6665 non-null   int64  
 3   NUID065253359           6665 non-null   object 
 4   1.1                     6665 non-null   object 
 5   071623 113711 101       6665 non-null   object 
 6   11:37 AM   07/16/2023   6665 non-null   object 
 7   2                       6663 non-null   object 
 8   1.2                     6663 non-null   float64
 9   01:37 PM   07/16/2023   6665 non-null   object 
 10  CreditCard              6665 non-null   object 
 11  F                       6665 non-null   object 
 12  41.08                   6665 non-null   float64
 13  Unnamed: 13             140 non-null    object 
 14  0.00                    6665 non-null   

In [5]:
idx_to_keep = [0, 1, 6, 7, 8, 9, 12, 21, 23]

In [6]:
# create new dataframe with specified column indices
df1 = df.iloc[:, idx_to_keep].copy()

In [7]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6665 entries, 0 to 6664
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   101                    6665 non-null   int64  
 1   NK                     6665 non-null   object 
 2   11:37 AM   07/16/2023  6665 non-null   object 
 3   2                      6663 non-null   object 
 4   1.2                    6663 non-null   float64
 5   01:37 PM   07/16/2023  6665 non-null   object 
 6   41.08                  6665 non-null   float64
 7   35.11                  6665 non-null   float64
 8   5.97                   6665 non-null   float64
dtypes: float64(4), int64(1), object(4)
memory usage: 468.8+ KB


In [8]:
# Create a dictionary to rename the columns
column_names = [
    "room_num",
    "room_type",
    "checkin_timestamp",
    "stay_duration",
    "num_nights",
    "checkout_timestamp",
    "total_price",
    "price",
    "tax",
]

# Assign column names to the DataFrame
df1.columns = column_names

In [9]:
df1.sample(n=5)

,room_num,room_type,checkin_timestamp,stay_duration,num_nights,checkout_timestamp,total_price,price,tax
2370,106,K,07:31 PM 02/14/2023,2,1.0,09:31 PM 02/14/2023,30.0,25.64,4.36
97,109,K,03:44 AM 06/18/2023,4,1.0,07:44 AM 06/18/2023,55.0,47.01,7.99
3858,129,NQQ,11:00 AM 05/02/2023,3,1.0,02:00 PM 05/02/2023,35.0,29.91,5.09
5024,108,K,12:07 PM 01/13/2023,1,1.0,01:07 PM 01/13/2023,25.0,21.37,3.63
6186,107,K,04:10 PM 01/29/2023,N,1.0,11:00 AM 01/30/2023,70.0,59.83,10.17


In [10]:
# Convert checkin_timestamp and checkout_timestamp
def convert_string_to_datetime(df, column_name, new_column_name):
    # Create a datetime object from the string object.
    datetime_object = pd.to_datetime(
        df[column_name], errors="coerce", format="mixed")

    # Add the datetime object to the DataFrame as a new column.
    new_datetime_column = f"{new_column_name}_datetime"
    df[new_datetime_column] = datetime_object

    # drop old timestamp columns
    df = df.drop([column_name], axis=1)

    return df

In [11]:
df1 = convert_string_to_datetime(df1.copy(), "checkin_timestamp", "checkin")
df1 = convert_string_to_datetime(df1.copy(), "checkout_timestamp", "checkout")

In [12]:
# drop old timestamp columns
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6665 entries, 0 to 6664
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   room_num           6665 non-null   int64         
 1   room_type          6665 non-null   object        
 2   stay_duration      6663 non-null   object        
 3   num_nights         6663 non-null   float64       
 4   total_price        6665 non-null   float64       
 5   price              6665 non-null   float64       
 6   tax                6665 non-null   float64       
 7   checkin_datetime   6665 non-null   datetime64[ns]
 8   checkout_datetime  6665 non-null   datetime64[ns]
dtypes: datetime64[ns](2), float64(4), int64(1), object(2)
memory usage: 468.8+ KB


In [13]:
# Convert the column to a categorical type
def convert_dtypes(df, column_name, data_type):
    df[column_name] = df[column_name].astype(data_type)

    return df

In [14]:
# convert object dtypes to category dtype and room_num to category dtype
category_features = ["room_num",
                     "room_type",
                     "stay_duration"]

data_type = "category"

for category in category_features:
    print(category)
    df1 = convert_dtypes(df1.copy(), category, data_type)

# print(df1.dtypes)

room_num
room_type
stay_duration


In [15]:
df1.sample()

,room_num,room_type,stay_duration,num_nights,total_price,price,tax,checkin_datetime,checkout_datetime
3995,117,K,2,1.0,35.0,29.91,5.09,2023-07-01 13:31:00,2023-07-01 15:31:00


In [16]:
# Subtracts two datetime columns in a Pandas DataFrame and calculates the difference into a day and hours column.
def subtract_datetime_columns(df, column_1, column_2):
    # Subtract the two datetime columns.
    # Compare the two datetime columns.
    greater_datetime_column = df[column_1].where(
        df[column_1] > df[column_2], df[column_2]
    )
    least_datetime_column = df[column_1].where(
        df[column_1] <= df[column_2], df[column_2]
    )

    # Subtract the greater datetime column by the least datetime column.
    difference = greater_datetime_column - least_datetime_column
    # difference = df[time1] - df[time0]

    # Convert the difference to a timedelta object.
    difference_timedelta = difference.dt.total_seconds()

    # Calculate the day and hours difference.
    day_difference = difference_timedelta // (24 * 60 * 60)
    hours_difference = (difference_timedelta % (24 * 60 * 60)) // (60 * 60)

    # Add the day and hours difference columns to the DataFrame.
    df["days"] = day_difference
    df["hours"] = hours_difference

    return df

In [17]:
df1 = subtract_datetime_columns(
    df1.copy(), "checkin_datetime", "checkout_datetime")

df1.sample(n=5)

,room_num,room_type,stay_duration,num_nights,total_price,price,tax,checkin_datetime,checkout_datetime,days,hours
609,120,K,N,1.0,0.0,0.00,0.00,2023-03-19 19:09:00,2023-03-20 11:00:00,0.0,15.0
3568,102,NK,1,1.0,35.0,29.91,5.09,2023-06-11 17:22:00,2023-06-11 18:22:00,0.0,1.0
5841,107,NK,N,2.0,0.0,0.00,0.00,2023-02-08 16:28:00,2023-02-10 11:00:00,1.0,18.0
6609,116,K,N,1.0,70.0,59.83,10.17,2023-04-30 21:48:00,2023-05-01 11:00:00,0.0,13.0
701,104,NK,2,1.0,40.0,34.19,5.81,2023-09-25 19:02:00,2023-09-25 21:02:00,0.0,2.0
